
## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19

In [2]:
import pandas as pd

In [27]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

In [28]:
len(df.columns)

19

### Answer: 19

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59
* 52.59
* 62.59

In [29]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [30]:
df.duration.std()

np.float64(42.59435124195458)

### Answer: 42.59

## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%


In [31]:
duration_old_count = df.duration.count()

In [32]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [33]:
(df.duration.count()/ duration_old_count) * 100


np.float64(98.1220282212598)

### Answer: 98.2%

## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715


In [37]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

In [39]:
train_dicts = df[categorical + numerical].to_dict(orient='records')

In [42]:
from sklearn.feature_extraction import DictVectorizer

In [43]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [44]:
X_train.shape

(3009173, 516)

### Answer: 515

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters, where duration is the response variable
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64


In [46]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [47]:
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

7.658398620393997

### Answer: 7.64

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81

In [50]:
df2 = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')

df2['duration'] = df2.tpep_dropoff_datetime - df2.tpep_pickup_datetime
df2.duration = df2.duration.apply(lambda td: td.total_seconds() / 60)

df2 = df2[(df2.duration >= 1) & (df2.duration <= 60)]

df2[categorical] = df2[categorical].astype(str)


In [51]:
train_dicts2 = df2[categorical + numerical].to_dict(orient='records')

dv2 = DictVectorizer()
X_train2 = dv2.fit_transform(train_dicts2)

target2 = 'duration'
y_train2 = df2[target2].values

lr2 = LinearRegression()
lr2.fit(X_train2, y_train2)

y_pred2 = lr2.predict(X_train2)

root_mean_squared_error(y_train2, y_pred2)

7.787516590658121

### Answer: 7.81